# BPMN Version Differ

This notebook demonstrates how to compare different versions of a BPMN process definition using `bpmn-js-differ`.

It uses:
- **operaton.py** - To fetch process definitions from the Operaton REST API
- **bpmn-js-differ** - To compute differences between BPMN diagrams
- **jupyterlab-bpmn** - To render the BPMN diagram with highlighted changes

> **Note on async limitations**: Due to limitations with async/await in ipywidgets callbacks
> in Pyodide/JupyterLite, this notebook pre-loads all data upfront and uses only synchronous
> operations in widget callbacks. See AGENTS.md for details.

## Setup and Data Loading

Initialize the required JavaScript libraries and pre-fetch all process definitions.
This cell performs all async operations upfront so that widget callbacks can be synchronous.

In [ ]:
import operaton
from operaton import Operaton

# Load environment variables from localStorage
await operaton.load_env()

# Load the required JavaScript libraries (async - must be done upfront)
await operaton.load_bpmn_moddle()
await operaton.load_bpmn_js_differ()

print("Libraries loaded!")

# Fetch all process definitions grouped by key
all_definitions = Operaton.get('/process-definition?latestVersion=false')

# Group by key
definitions_by_key = {}
for defn in all_definitions:
    key = defn['key']
    if key not in definitions_by_key:
        definitions_by_key[key] = []
    definitions_by_key[key].append(defn)

# Sort versions within each key
for key in definitions_by_key:
    definitions_by_key[key].sort(key=lambda d: d['version'])

# Filter to only keys with multiple versions
keys_with_versions = [k for k, v in definitions_by_key.items() if len(v) > 1]

if not keys_with_versions:
    print("No process definitions with multiple versions found.")
    print("Available process definitions:")
    for key, defs in definitions_by_key.items():
        print(f"  - {key}: {len(defs)} version(s)")
else:
    print(f"Found {len(keys_with_versions)} process definition(s) with multiple versions:")
    for key in keys_with_versions:
        print(f"  - {key}: {len(definitions_by_key[key])} versions")

print("\nPre-fetching and parsing BPMN XML for all versions...")

# Pre-fetch and parse all BPMN XML (async operations done upfront)
# This cache stores parsed BPMN definitions that can be used synchronously later
xml_cache = {}  # id -> raw XML string
parsed_cache = {}  # id -> parsed result with rootElement

async def prefetch_all():
    for key in keys_with_versions:
        for defn in definitions_by_key[key]:
            defn_id = defn['id']
            # Fetch XML (sync operation via XMLHttpRequest)
            response = Operaton.get(f'/process-definition/{defn_id}/xml')
            xml = response['bpmn20Xml']
            xml_cache[defn_id] = xml
            # Parse BPMN (async operation - must be done upfront)
            parsed = await operaton.parse_bpmn(xml)
            parsed_cache[defn_id] = parsed
            print(f"  ✓ Loaded {key} v{defn['version']}")

await prefetch_all()
print(f"\nSetup complete! Pre-loaded {len(xml_cache)} versions.")

## Select and Compare Versions

Choose a process definition and select versions to compare.
All operations in the widget callbacks are synchronous since data was pre-loaded above.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create widgets for process and version selection
process_dropdown = widgets.Dropdown(
    options=[(f"{k} ({len(definitions_by_key[k])} versions)", k) for k in keys_with_versions],
    description='Process:',
    style={'description_width': 'initial'}
)

old_version_dropdown = widgets.Dropdown(
    options=[],
    description='Old version:',
    style={'description_width': 'initial'}
)

new_version_dropdown = widgets.Dropdown(
    options=[],
    description='New version:',
    style={'description_width': 'initial'}
)

compare_button = widgets.Button(
    description='Compare Versions',
    button_style='primary',
    icon='search'
)

output = widgets.Output()

# Store comparison results for use in rendering
comparison_result = {}

def update_versions(change):
    """Update version dropdowns when process selection changes."""
    key = change['new']
    if key:
        versions = definitions_by_key[key]
        version_options = [(f"v{d['version']}: {d['id'][:30]}...", d['id']) for d in versions]
        old_version_dropdown.options = version_options
        new_version_dropdown.options = version_options
        # Default: compare oldest with newest
        if len(versions) >= 2:
            old_version_dropdown.value = versions[0]['id']
            new_version_dropdown.value = versions[-1]['id']

def on_compare_click(button):
    """Perform the comparison when button is clicked (SYNCHRONOUS)."""
    global comparison_result
    
    with output:
        clear_output()
        
        old_id = old_version_dropdown.value
        new_id = new_version_dropdown.value
        
        if old_id == new_id:
            print("Please select two different versions to compare.")
            return
        
        print(f"Comparing versions...")
        
        # Get pre-parsed definitions from cache (synchronous!)
        old_parsed = parsed_cache[old_id]
        new_parsed = parsed_cache[new_id]
        
        # Compute diff using synchronous function with pre-parsed definitions
        diff = operaton.compare_bpmn_definitions(
            old_parsed.rootElement,
            new_parsed.rootElement
        )
        
        print(f"\n=== Comparison Results ===")
        print(f"Added elements: {len(diff.added_ids)}")
        for elem_id in diff.added_ids:
            print(f"  + {elem_id}")
        
        print(f"\nRemoved elements: {len(diff.removed_ids)}")
        for elem_id in diff.removed_ids:
            print(f"  - {elem_id}")
        
        print(f"\nChanged elements: {len(diff.changed_ids)}")
        for elem_id in diff.changed_ids:
            print(f"  ~ {elem_id}")
        
        print(f"\nLayout changed: {len(diff.layout_changed_ids)}")
        for elem_id in diff.layout_changed_ids:
            print(f"  # {elem_id}")
        
        # Store results for rendering
        comparison_result = {
            'old_xml': xml_cache[old_id],
            'new_xml': xml_cache[new_id],
            'diff': diff,
            'added': diff.added_ids,
            'removed': diff.removed_ids,
            'changed': diff.changed_ids,
            'layout_changed': diff.layout_changed_ids
        }
        
        print("\n✓ Comparison complete. Run the next cell to visualize.")

process_dropdown.observe(update_versions, names='value')
compare_button.on_click(on_compare_click)

# Initialize versions for the first process
if keys_with_versions:
    update_versions({'new': keys_with_versions[0]})

# Display the widgets
display(widgets.VBox([
    process_dropdown,
    widgets.HBox([old_version_dropdown, new_version_dropdown]),
    compare_button,
    output
]))

## Visualize Changes

After clicking "Compare Versions" above, run the cells below to render the BPMN diagrams with changes highlighted:
- **Green**: Added elements
- **Yellow**: Changed elements  
- **Blue**: Layout changed only
- **Red**: Removed elements (shown on old diagram)

In [ ]:
from IPython.display import display
import json

if not comparison_result:
    print("Please run the comparison first (click 'Compare Versions' button above).")
else:
    # Build colors map for highlighting changes
    colors = {}
    
    # Added elements - green
    for elem_id in comparison_result['added']:
        colors[elem_id] = {'stroke': '#22863a', 'fill': '#dcffe4'}
    
    # Changed elements - yellow/orange
    for elem_id in comparison_result['changed']:
        colors[elem_id] = {'stroke': '#b08800', 'fill': '#fff5b1'}
    
    # Layout changed - blue
    for elem_id in comparison_result['layout_changed']:
        colors[elem_id] = {'stroke': '#0366d6', 'fill': '#dbedff'}
    
    # Prepare the BPMN display data
    data = json.dumps({
        'style': {'height': '500px'},
        'colors': colors,
        'zoom': 0.8
    })
    
    print("New version with changes highlighted:")
    display({
        'application/bpmn+xml': comparison_result['new_xml'],
        'application/bpmn+json': data
    }, raw=True)

In [ ]:
# Also show the old version with removed elements highlighted
if comparison_result and comparison_result['removed']:
    # Build colors for removed elements (shown on old diagram)
    old_colors = {}
    for elem_id in comparison_result['removed']:
        old_colors[elem_id] = {'stroke': '#cb2431', 'fill': '#ffeef0'}
    
    old_data = json.dumps({
        'style': {'height': '500px'},
        'colors': old_colors,
        'zoom': 0.8
    })
    
    print("Old version with removed elements highlighted:")
    display({
        'application/bpmn+xml': comparison_result['old_xml'],
        'application/bpmn+json': old_data
    }, raw=True)
elif comparison_result:
    print("No elements were removed between versions.")

## Summary

Display a summary of all changes.

In [ ]:
if comparison_result:
    diff = comparison_result['diff']
    
    total_changes = (
        len(comparison_result['added']) +
        len(comparison_result['removed']) +
        len(comparison_result['changed']) +
        len(comparison_result['layout_changed'])
    )
    
    print("="*50)
    print(f"Total changes detected: {total_changes}")
    print("="*50)
    print(f"  🟢 Added:          {len(comparison_result['added'])}")
    print(f"  🔴 Removed:        {len(comparison_result['removed'])}")
    print(f"  🟡 Changed:        {len(comparison_result['changed'])}")
    print(f"  🔵 Layout changed: {len(comparison_result['layout_changed'])}")
else:
    print("Run the comparison first.")